# Problem Statement
Implementation of RSA

#Elliptic Curve Class

In [1]:
INF_POINT = None

class EllipticCurve:
  
	def __init__(self, p, a, b):
		self.p = p
		self.a = a
		self.b = b


	def addition(self, P1, P2):
		if P1 == INF_POINT:
			return P2
		if P2 == INF_POINT:
			return P1

		(x1, y1) = P1
		(x2, y2) = P2

		if self.equal_modp(x1, x2) and self.equal_modp(y1, -y2):
			return INF_POINT

		if self.equal_modp(x1, x2) and self.equal_modp(y1, y2):
			u = self.reduce_modp((3 * x1 * x1 + self.a) * self.inverse_modp(2 * y1))
		else:
			u = self.reduce_modp((y1 - y2) * self.inverse_modp(x1 - x2))

		v = self.reduce_modp(y1 - u * x1)
		x3 = self.reduce_modp(u * u - x1 - x2)
		y3 = self.reduce_modp(-u * x3 - v)
		return (x3, y3)


	def multiple(self, k, P):
		Q = INF_POINT
		if k == 0:
			return Q
		while k != 0:
			if k & 1 != 0:
				Q = self.addition(Q, P)
			P = self.addition(P, P)
			k >>= 1
		return Q


	def is_point_on_curve(self, x, y):
		return self.equal_modp(y * y, x * x * x + self.a * x + self.b)


	# helper functions

	def reduce_modp(self, x):
		return x % self.p


	def equal_modp(self, x, y):
		return self.reduce_modp(x - y) == 0


	def inverse_modp(self, x):
		if self.reduce_modp(x) == 0:
			return None
		return pow(x, p - 2, p)



In [2]:
#Elliptic Curve
p = 26959946667150639794667015087019630673557916260026308143510066298881
a = -3
b = 18958286285566608000408668544493926415504680968679321075787234672564
P224 = EllipticCurve(p, a, b)

#Generator Point P
Px = 19277929113566293071110308034699488026831934219452440156649784352033
Py = 19926808758034470970197974370888749184205991990603949537637343198772
P = (Px, Py)

#limit
n = 26959946667150639794667015087019625940457807714424391721682722368061

# Key generation (Receiver Side)

In [3]:
# Q is the public key
# d is the private key

def key_generation():
  from random import randint
  d = randint(1, n-1)
  Q = P224.multiple(d, P)
  return Q, d

Q, d = key_generation()

#Encryption and Decryption of a single character

In [4]:
def encryption(m, Q):

  from random import randint

  # Encode the message as a point
  M = (ord(m), 1)

  # Random Integer
  k = randint(1, n-1)

  #Encryption using Receiver's Public Key

  # C1 = k*P
  C1 = P224.multiple(k, P)

  # C2 = M + k*Q
  C2 = P224.multiple(k, Q)
  C2 = (C2[0] + M[0], C2[1] + M[1])
  
  return C1, C2

C1, C2 = encryption('F', Q)

In [5]:
def decryption(C1, C2, d):
  De = P224.multiple(d, C1)
  De = (C2[0] - De[0], C2[1] - De[1])
  return chr(De[0])

de = decryption(C1, C2, d)
de

'F'

# Encryption and Decryption of a string

In [6]:
def encrypt_text(message, Q):
  C1_arr, C2_arr = [], []
  for character in message:
    C1, C2 = encryption(character, Q)
    C1_arr.append(C1)
    C2_arr.append(C2)
  return C1_arr, C2_arr

C1_arr, C2_arr = encrypt_text('Sos1!', Q)

In [7]:
def decrypt_text(C1_arr, C2_arr, d):
  decrypted_text = ''
  for C1, C2 in zip(C1_arr, C2_arr):
    de = decryption(C1, C2, d)
    decrypted_text += de
  return decrypted_text

decrypt_text(C1_arr, C2_arr, d)

'Sos1!'